In [ ]:
import numpy as np
import pandas as pd 
import altair as alt
import re 
import ntpath
from commons import data_processing
from commons import common_objects as co
import matplotlib.pyplot as plt
from venn import venn
from scipy import stats, signal
from matplotlib_venn import venn2
import warnings

warnings.filterwarnings('ignore')
alt.data_transformers.disable_max_rows()

In [ ]:
# pgc = data_processing.get_files(r'./data/DIA', exts=['parquet'])
# rplc = data_processing.get_files(r'./data/DIA/', exts=['parquet'])
files = data_processing.get_files(r'./data/DIA', exts=['parquet'])
column_colors = alt.Color('column:N',
    scale=alt.Scale(
        domain=["PGC", "RPLC"],
        range=["#D56E3B", "#58728C"]
    ))


In [ ]:
df = pd.DataFrame()

for file in files:

    _df = pd.read_parquet(file)
    col = ntpath.dirname(file).split('\\')[-1]
    _df.loc[:, 'column'] = col 

    _df.loc[:, 'sample'] = _df.Run.map(lambda x: x.split('_')[-3])
    _df.loc[:, 'tech_rep'] = _df.Run.map(lambda x: x.split('_')[-1])

    df = pd.concat([df, _df])
    df.reset_index(inplace=True, drop=True)

In [ ]:
g = df.groupby(['sample', 'column', 'Modified.Sequence']).mean()

samples = df['sample'].unique()
columns = df['column'].unique()

fig, axs = plt.subplots(1, 2, figsize=(10,5))
i = 0
for s in samples:
    overlap = {}
    for c in columns:
        small = g.loc[(s, c), :]
        small.reset_index(inplace=True)
        overlap[c] = set(small["Modified.Sequence"].tolist())
    venn(overlap, ax=axs[i])
    axs[i].set_title(s)

    i += 1

g = df.groupby(['column', 'Modified.Sequence']).mean()
overlap = {}
for c in columns:
    small = g.loc[(c), :]
    small.reset_index(inplace=True)
    overlap[c] = set(small["Modified.Sequence"].tolist())
venn(overlap)


In [ ]:
d = df[(df["Proteotypic"]==1)&(df["Protein.Q.Value"]<=0.01)]
d = d.groupby(['sample', 'column', 'Protein.Ids']).mean()

samples = df['sample'].unique()

fig, axs = plt.subplots(1, 2)
i = 0
for s in samples:
    overlap = {}
    for c in columns:
        small = d.loc[(s, c), :]
        small.reset_index(inplace=True)
        overlap[c] = set(small["Protein.Ids"].tolist())
    venn(overlap, ax=axs[i])
    axs[i].set_title(s)

    i += 1

d = df[(df["Proteotypic"]==1)&(df["Protein.Q.Value"]<=0.01)]
d = d.groupby(['column', 'Protein.Ids']).mean()

overlap = {}
for c in columns:
    small = d.loc[(c), :]
    small.reset_index(inplace=True)
    overlap[c] = set(small["Protein.Ids"].tolist())
venn(overlap)